In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from sklearn import metrics
import spacy
from sklearn.linear_model import LogisticRegression
import string

Загрузка данные сета.

In [2]:
data = pd.read_csv('IMDB Dataset.csv', sep=",")

Оцениваем что в дата-сете есть, и с чем прийдется иметь дело.

In [3]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Cмотрим насколько однородны данные и сколько пропущенных элементов в тех или иных столбцах.

In [4]:
data.count()

review       50000
sentiment    50000
dtype: int64

В процентном соотношении.

In [5]:
for col in data.columns:
    pct_missing = np.mean(data[col].isnull())
    print(f'{col} - {round(pct_missing * 100, 5)}%')

review - 0.0%
sentiment - 0.0%


In [6]:
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


Видов отзывово только два (позитивные и негативные), по этому заменим таргет.

In [7]:
data["sentiment"] = np.where(data["sentiment"] == "positive", 1, 0)

In [8]:
data['review'] = data["review"].str.replace(r'<.*>',"")

C:\Users\Monya\AppData\Local\Temp\ipykernel_9512\3582704259.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['review'] = data["review"].str.replace(r'<.*>',"")


In [9]:
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_md')

In [10]:
punctuations = string.punctuation
stop_words = ENGLISH_STOP_WORDS

In [11]:
def spacy_tokenizer(odjects):
    token = nlp(str(odjects))
    token = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in token ]
    token = [ word for word in token if word not in stop_words and word not in punctuations ]
    return token

In [12]:
target = data["sentiment"]
data.drop(columns=["sentiment"],inplace=True)
data = data["review"]

In [13]:
datasets = {"train": {}, "test": {}}
datasets["train"]["data"], datasets["test"]["data"], datasets["train"]["target"], datasets["test"][
    "target"] = train_test_split(data, target, test_size=0.1, shuffle=True)
print('Training Data Shape:', datasets["train"]["data"].shape)
print('Testing Data Shape:', datasets["test"]["data"].shape)

Training Data Shape: (45000,)
Testing Data Shape: (5000,)


In [14]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [15]:
print(datasets["train"]["data"].shape,datasets["train"]["target"].shape)

(45000,) (45000,)


In [16]:
classifier = LogisticRegression()

# Create pipeline using TF-IDF
pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(datasets["train"]["data"],datasets["train"]["target"])

D:\Hilell\задания\venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function spacy_tokenizer at 0x000001932D109E10>)),
                ('classifier', LogisticRegression(max_iter=20000))])

Создаём предикт.

In [17]:
prediction = pipe.predict(datasets["test"]["data"])

In [18]:
print(len(prediction))

5000


In [19]:
matrix = metrics.confusion_matrix(datasets["test"]["target"], prediction,labels=[0,1])
matrix = pd.DataFrame(matrix,columns=["Предикт 1","Предик 0"],index=["Факт 1","Факт 0"])
matrix

,Предикт 1,Предик 0
Факт 1,2126,385
Факт 0,320,2169


In [20]:
accuracy= metrics.accuracy_score(datasets["test"]["target"], prediction)
balanced_accuracy = metrics.balanced_accuracy_score(datasets["test"]["target"], prediction)
precision = metrics.precision_score(datasets["test"]["target"], prediction)
recall = metrics.recall_score(datasets["test"]["target"], prediction)

print("Presicion for Logistic Regression: ",round(precision,2))
print("Recall for Logistic Regression: ",round(recall,2))
print("Accurracy for Logistic Regression: ",round(accuracy,2))
print("Balanced accurracy for Logistic Regression: ",round(balanced_accuracy,2))

Presicion for Logistic Regression:  0.85
Recall for Logistic Regression:  0.87
Accurracy for Logistic Regression:  0.86
Balanced accurracy for Logistic Regression:  0.86


Все метрики модели Logistic Regression остановились на достраточно высоком уровне. Что свидетельствует о правильности проведённых преобразований и эффективности выбранных методов.